## 학습 알고리즘

- 유저의 행동데이터를 받아와서 기초 알고리즘의 가중치들 수정
- 메인 알고리즘을 작성하여 유저들의 속성과 그 행동 데이터들 만으로 가중치 설정
    - 유저의 향후 행동 데이터를 계속 학습

### 행동 데이터 (유저 선택의 결과)

- 유저가 일정을 변경하는 버튼을 따로 만들고, 이 버튼(일정 확정)이 푸시 되었을 때와 그 전을 비교해서 가중치에 수정을 가한다
- 유저가 자유롭게 일정을 드래그 & 드롭으로 수정할 수 있게 만들고, 수정된 것에 대한 가중치를 수집한다 (이런 경우엔 세션 종료 시 변경 여부를 가져와서 가중치 수정을 가하는 방법이겠지 ??)
- todo를 루틴과 루틴이 아닌 것으로 구분

- 유저가 많이 수행하는 todo의 카테고리를 잡고, 해당 카테고리에서 벗어나는 카테고리를 주의 카테고리로 설정

### 랜덤 포레스트 모델

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

In [10]:
# 가독성을 위해 지수 표현(e) 변환
pd.options.display.float_format = '{:.5f}'.format

In [11]:
file_path = './refer/output/'

In [12]:
# 데이터 로딩
data = pd.read_csv(f'{file_path}survey_data.csv')

In [13]:
# 범주형 데이터를 원-핫 인코딩
categorical_features = ['gender', 'mbti', 'job']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# 전처리 파이프라인 설정
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# 타겟 변수와 피처 분리
X = data[['age', 'gender', 'mbti', 'job']]
y = data[['work', 'edu', 'free_time', 'health', 'chores']]

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 파이프라인 구성
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# 모델 학습
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_test)

# 예측 결과를 데이터프레임으로 변환
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)

# MSE 계산 결과를 데이터프레임으로 변환
mse = mean_squared_error(y_test, y_pred)
mse_df = pd.DataFrame([mse], columns=['MSE'])

# 결과 출력
print(mse_df)
print(y_pred_df)

      MSE
0 0.00000
          edu  free_time  health  chores
0     0.82500    0.80000 0.90000 0.67500
1     0.71875    0.96875 0.96875 0.50000
2     0.71875    0.96875 0.96875 0.50000
3     0.82500    0.80000 0.90000 0.67500
4     0.71875    0.96875 0.96875 0.50000
...       ...        ...     ...     ...
23071 0.71875    0.96875 0.96875 0.50000
23072 0.82500    0.80000 0.90000 0.67500
23073 0.59374    0.56250 0.49998 0.34376
23074 0.71875    0.96875 0.96875 0.50000
23075 0.82500    0.80000 0.90000 0.67500

[23076 rows x 4 columns]


### 그라디언트 부스팅 회귀 모델

#### LightGBM 빌드 **(트리 조정, 학습률 조정 필요)

In [24]:
# 라이브러리 로드
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

In [26]:
# 데이터 로딩
data = pd.read_csv(f'{file_path}survey_data.csv')

In [29]:
# 범주형 데이터를 원-핫 인코딩
categorical_features = ['gender', 'mbti', 'job']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# 전처리 파이프라인 설정
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# 타겟 변수와 피처 분리
X = data[['age', 'gender', 'mbti', 'job']]
y = data[['work', 'edu', 'free_time', 'health', 'chores']]

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 전처리
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# 각 타겟에 대한 모델 훈련 및 예측
mse = []
predictions = []
for target in y.columns:
    model = LGBMRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train[target])
    y_pred = model.predict(X_test)
    mse.append(mean_squared_error(y_test[target], y_pred))
    predictions.append(y_pred)

# 예측 결과를 데이터프레임으로 변환
y_pred_df = pd.DataFrame(zip(*predictions), columns=y.columns)

# MSE 결과 출력
for idx, target in enumerate(y.columns):
    print(f'MSE for {target}: {format(mse[idx], ".4f")}')

# 전체 MSE 계산 및 출력
overall_mse = mean_squared_error(y_test, y_pred_df)
print("Overall MSE:", format(overall_mse, ".4f"))

# 예측 결과 데이터프레임 출력, 지수 표현식 없이
print(y_pred_df.to_string(index=False, header=True, float_format="{:0.4f}".format))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000528 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 42
[LightGBM] [Info] Number of data points in the train set: 92304, number of used features: 17
[LightGBM] [Info] Start training from score 0.764643
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

In [ ]:
# 이 코드 적용해서 하이퍼 파라미터 수정 필요
'''
for target in y.columns:
    model = LGBMRegressor(
        n_estimators=200,  # 트리의 수 증가
        random_state=42,
        max_depth=10,  # 트리의 최대 깊이 증가
        min_data_in_leaf=20,  # 한 리프가 가지는 최소 데이터 수
        learning_rate=0.05  # 학습률 감소
    )
    model.fit(X_train, y_train[target])
    y_pred = model.predict(X_test)
    mse.append(mean_squared_error(y_test[target], y_pred))
    predictions.append(y_pred)
    print(f'MSE for {target}: {format(mean_squared_error(y_test[target], y_pred), ".4f")}')

'''

#### XGBoost 빌드

In [22]:
# 라이브러리 로드
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error

MSE for work: 7.041376330285435e-11
MSE for edu: 8.463449225554134e-11
MSE for free_time: 3.459870447017011e-10
MSE for health: 5.435218861354711e-10
MSE for chores: 7.723942988605882e-11
Overall MSE: 0.0000
  work    edu  free_time  health  chores
0.9500 0.8250     0.8000  0.9000  0.6750
0.6250 0.7187     0.9688  0.9687  0.5000
0.6250 0.7188     0.9687  0.9687  0.5000
0.9500 0.8250     0.8000  0.9000  0.6750
0.6250 0.7188     0.9687  0.9687  0.5000
0.7188 0.5938     0.5625  0.5000  0.3438
0.7188 0.5938     0.5625  0.5000  0.3438
0.9500 0.8250     0.8000  0.9000  0.6750
0.9500 0.8250     0.8000  0.9000  0.6750
0.6250 0.7188     0.9687  0.9687  0.5000
0.9500 0.8250     0.8000  0.9000  0.6750
0.7188 0.5938     0.5625  0.5000  0.3438
0.9500 0.8250     0.8000  0.9000  0.6750
0.7188 0.5938     0.5625  0.5000  0.3438
0.6250 0.7187     0.9687  0.9687  0.5000
0.6250 0.7187     0.9688  0.9687  0.5000
0.9500 0.8250     0.8000  0.9000  0.6750
0.6250 0.7187     0.9688  0.9687  0.5000
0.7188 0.5938

In [ ]:
# 데이터 로딩
data = pd.read_csv(f'{file_path}survey_data.csv')

In [ ]:
# 범주형 데이터를 원-핫 인코딩
categorical_features = ['gender', 'mbti', 'job']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# 전처리 파이프라인 설정
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# 타겟 변수와 피처 분리
X = data[['age', 'gender', 'mbti', 'job']]
y = data[['work', 'edu', 'free_time', 'health', 'chores']]

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 전처리 및 변환
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# 각 타겟에 대한 모델 훈련 및 예측 결과 저장
predictions = []
for i, target in enumerate(y.columns):
    model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
    model.fit(X_train, y_train.iloc[:, i])
    y_pred = model.predict(X_test)
    predictions.append(y_pred)
    print(f'MSE for {target}: {mean_squared_error(y_test.iloc[:, i], y_pred)}')

# 예측 결과를 데이터프레임으로 변환
y_pred_df = pd.DataFrame(predictions).T  # Transpose to align with y_test's shape
y_pred_df.columns = y_test.columns

# 예측 데이터프레임과 실제 데이터프레임의 MSE 계산
mse = mean_squared_error(y_test, y_pred_df)
print("Overall MSE:", format(mse, ".4f"))

# 예측 결과 데이터프레임 출력, 지수 표현식 없이
print(y_pred_df.to_string(index=False, header=True, float_format="{:0.4f}".format))

### 딥러닝 모델

#### TensorFlow 빌드

In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [31]:
# 데이터 로딩
data = pd.read_csv(f'{file_path}survey_data.csv')

In [34]:
# 범주형 데이터를 원-핫 인코딩
categorical_features = ['gender', 'mbti', 'job']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# 전처리 파이프라인 설정
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# 타겟 변수와 피처 분리
X = data[['age', 'gender', 'mbti', 'job']]
y = data[['work', 'edu', 'free_time', 'health', 'chores']]

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 전처리 및 변환
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [35]:
# 딥러닝 모델 구성
model = Sequential([
    Dense(128, activation='relu', input_dim=X_train.shape[1]),
    Dense(64, activation='relu'),
    Dense(5)  # 출력 레이어: 5개 타겟 변수
])

# 모델 컴파일
model.compile(optimizer='adam', loss='mse')

# 모델 학습
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50


c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2308/2308 ━━━━━━━━━━━━━━━━━━━━ 3s 884us/step - loss: 0.1436 - val_loss: 0.0042
Epoch 2/50
2308/2308 ━━━━━━━━━━━━━━━━━━━━ 2s 914us/step - loss: 0.0041 - val_loss: 0.0041
Epoch 3/50
2308/2308 ━━━━━━━━━━━━━━━━━━━━ 2s 880us/step - loss: 0.0036 - val_loss: 0.0033
Epoch 4/50
2308/2308 ━━━━━━━━━━━━━━━━━━━━ 2s 863us/step - loss: 0.0034 - val_loss: 0.0031
Epoch 5/50
2308/2308 ━━━━━━━━━━━━━━━━━━━━ 2s 876us/step - loss: 0.0029 - val_loss: 0.0026
Epoch 6/50
2308/2308 ━━━━━━━━━━━━━━━━━━━━ 2s 854us/step - loss: 0.0024 - val_loss: 0.0016
Epoch 7/50
2308/2308 ━━━━━━━━━━━━━━━━━━━━ 2s 870us/step - loss: 0.0013 - val_loss: 6.8667e-05
Epoch 8/50
2308/2308 ━━━━━━━━━━━━━━━━━━━━ 2s 851us/step - loss: 6.9414e-05 - val_loss: 5.1433e-06
Epoch 9/50
2308/2308 ━━━━━━━━━━━━━━━━━━━━ 2s 899us/step - loss: 1.0154e-04 - val_loss: 6.8259e-05
Epoch 10/50
2308/2308 ━━━━━━━━━━━━━━━━━━━━ 2s 862us/step - loss: 0.0012 - val_loss: 1.4622e-06
Epoch 11/50
2308/2308 ━━━━━━━━━━━━━━━━━━━━ 2s 843us/step - loss: 1.3244e-04 - val_loss

In [37]:
# 예측 및 평가
y_pred = model.predict(X_test)

# MSE 계산
mse = tf.keras.losses.MeanSquaredError()
mse_value = mse(y_test, y_pred).numpy()
print("MSE:", format(mse_value, ".4f"))


722/722 ━━━━━━━━━━━━━━━━━━━━ 1s 700us/step
MSE: 0.0000
